# Exporting / Importing trained MaskRCNN model


In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np
import imageio
import tensorflow as tf
from tensorflow.python.framework import graph_util
from keras import backend as K

from config import Config
from inference_config import inference_config
import model as modellib
import utils
import visualize
import matplotlib.pyplot as plt

%matplotlib inline 

c:\programdata\anaconda3\envs\maskrcnn\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

# Exporting model
After retrain the model using train.py, we export the model 

In [3]:
# Root directory of the project
ROOT_DIR = os.getcwd()
# Directory to trained model
train_log_dirpath = os.path.join(ROOT_DIR, "logs")
print(train_log_dirpath)
model_filepath = None# os.path.join(ROOT_DIR, "logs","trained_model","my_model.h5")
print(model_filepath)
# name of the pb file we want to output
filename = 'mask_rcnn_New.pb'

C:\Users\ADMIN\Mask\def_single\logs
None


In [4]:
class InferenceConfig(Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    NAME= "myConfig"
    GPU_COUNT = 1
    NUM_CLASSES = 2
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 512 
    IMAGE_MAX_DIM = 512 
    USE_MINI_MASK = False
    RESNET_ARCHITECTURE = "resnet50"
    DETECTION_MAX_INSTANCES = 50
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    TRAIN_ROIS_PER_IMAGE = 100
    STEPS_PER_EPOCH = 200
    VALIDATION_STEPS = 5

myconfig = InferenceConfig()
myconfig.display()


Configurations:
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        50
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           myConfig
NUM_CLASSES                    2
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING       

In [5]:
# Build the inference model
model = modellib.MaskRCNN(mode="inference",
                          config=inference_config,
                          model_dir=train_log_dirpath)
# Get path to saved weights. Either set a specific path or find last trained weights
model_filepath = model_filepath if model_filepath else model.find_last()[1]
#print(model_filepath)
# Load trained weights (fill in path to trained weights here)
assert model_filepath, "Provide path to trained weights"
model.load_weights(model_filepath, by_name=True)
print("Model loaded.")

Model loaded.


In [6]:
# Get keras model and save
model_keras= model.keras_model
# All new operations will be in test mode from now on.
K.set_learning_phase(0)
# Create output layer with customized names
num_output = 7
pred_node_names = ["detections", "mrcnn_class", "mrcnn_bbox", "mrcnn_mask",
                       "rois", "rpn_class", "rpn_bbox"]
pred_node_names = ["output_" + name for name in pred_node_names]
print(pred_node_names)

['output_detections', 'output_mrcnn_class', 'output_mrcnn_bbox', 'output_mrcnn_mask', 'output_rois', 'output_rpn_class', 'output_rpn_bbox']


In [7]:
pred = [tf.identity(model_keras.outputs[i], name = pred_node_names[i])
        for i in range(num_output)]
sess = K.get_session()
# Get the object detection graph
od_graph_def = graph_util.convert_variables_to_constants(sess,
                                                         sess.graph.as_graph_def(),
                                                         pred_node_names)

INFO:tensorflow:Froze 384 variables.
Converted 384 variables to const ops.


In [8]:
model_dirpath = os.path.dirname(model_filepath)
pb_filepath = os.path.join(model_dirpath, filename)
print('Saving frozen graph {} ...'.format(os.path.basename(pb_filepath)))

frozen_graph_path = pb_filepath
with tf.gfile.GFile(frozen_graph_path, 'wb') as f:
    f.write(od_graph_def.SerializeToString())
print('{} ops in the frozen graph.'.format(len(od_graph_def.node)))
#print('pb file saved at', model_dirpath)

Saving frozen graph mask_rcnn_New.pb ...
2450 ops in the frozen graph.


# Importing model
Now, we can load the model from the pb file and then use it to infere

In [9]:
def mold_inputs(images):
        """Takes a list of images and modifies them to the format expected
        as an input to the neural network.
        images: List of image matricies [height,width,depth]. Images can have
            different sizes.

        Returns 3 Numpy matricies:
        molded_images: [N, h, w, 3]. Images resized and normalized.
        image_metas: [N, length of meta data]. Details about each image.
        windows: [N, (y1, x1, y2, x2)]. The portion of the image that has the
            original image (padding excluded).
        """
        molded_images = []
        image_metas = []
        windows = []
        print('IMAGE_PADDING: ',inference_config.IMAGE_PADDING)
        for image in images:
            # Resize image to fit the model expected size
            # TODO: move resizing to mold_image()
            molded_image, window, scale, padding = utils.resize_image(
                image,
                min_dim=inference_config.IMAGE_MIN_DIM,
                max_dim=inference_config.IMAGE_MAX_DIM,
                padding=inference_config.IMAGE_PADDING)
            print(image.shape)
            print('Image resized at: ', molded_image.shape)
            print(window)
            print(scale)
            """Takes RGB images with 0-255 values and subtraces
                   the mean pixel and converts it to float. Expects image
                   colors in RGB order."""
            molded_image = mold_image(molded_image, inference_config)
            print('Image molded')
            #print(a)
            """Takes attributes of an image and puts them in one 1D array."""
            image_meta = compose_image_meta(
                0, image.shape, window,
                np.zeros([inference_config.NUM_CLASSES], dtype=np.int32))
            print('Meta of image prepared')
            # Append
            molded_images.append(molded_image)
            windows.append(window)
            image_metas.append(image_meta)
        # Pack into arrays
        molded_images = np.stack(molded_images)
        image_metas = np.stack(image_metas)
        windows = np.stack(windows)
        return molded_images, image_metas, windows

def mold_image(images, config):
    return images.astype(np.float32) - config.MEAN_PIXEL

def compose_image_meta(image_id, image_shape, window, active_class_ids):
    """Takes attributes of an image and puts them in one 1D array.

    image_id: An int ID of the image. Useful for debugging.
    image_shape: [height, width, channels]
    window: (y1, x1, y2, x2) in pixels. The area of the image where the real
            image is (excluding the padding)
    active_class_ids: List of class_ids available in the dataset from which
        the image came. Useful if training on images from multiple datasets
        where not all classes are present in all datasets.
    """
    meta = np.array(
        [image_id] +            # size=1
        list(image_shape) +     # size=3
        list(window) +          # size=4 (y1, x1, y2, x2) in image cooredinates
        list(active_class_ids)  # size=num_classes
    )
    return meta

def unmold_detections(detections, mrcnn_mask, image_shape, window):
    """Reformats the detections of one image from the format of the neural
    network output to a format suitable for use in the rest of the
    application.

    detections: [N, (y1, x1, y2, x2, class_id, score)]
    mrcnn_mask: [N, height, width, num_classes]
    image_shape: [height, width, depth] Original size of the image before resizing
    window: [y1, x1, y2, x2] Box in the image where the real image is excluding the padding.

        Returns:
        boxes: [N, (y1, x1, y2, x2)] Bounding boxes in pixels
        class_ids: [N] Integer class IDs for each bounding box
        scores: [N] Float probability scores of the class_id
        masks: [height, width, num_instances] Instance masks
        """
    # How many detections do we have?
    # Detections array is padded with zeros. Find the first class_id == 0.
    zero_ix = np.where(detections[:, 4] == 0)[0]
    N = zero_ix[0] if zero_ix.shape[0] > 0 else detections.shape[0]
    print('Number of detections: ',N)
    print('Window: ',window)
    # Extract boxes, class_ids, scores, and class-specific masks
    boxes = detections[:N, :4]
    print('boxes',boxes.shape,' ',boxes)
    class_ids = detections[:N, 4].astype(np.int32)
    print('Class_ids: ',class_ids.shape,' ',class_ids)
    scores = detections[:N, 5]
    print('Scores: ',scores.shape,' ',scores)
    masks = mrcnn_mask[np.arange(N), :, :, class_ids]
    print('Masks: ',masks.shape)# masks)
    # Compute scale and shift to translate coordinates to image domain.
    print(image_shape[0])
    print(window[2] - window[0])
    h_scale = image_shape[0] / (window[2] - window[0])
    print('h_scale: ',h_scale)
    w_scale = image_shape[1] / (window[3] - window[1])
    print('w_scale: ',w_scale)
    scale = min(h_scale, w_scale)
    shift = window[:2]  # y, x
    print('shift: ',shift)
    scales = np.array([scale, scale, scale, scale])
    print('scales: ',scales)
    shifts = np.array([shift[0], shift[1], shift[0], shift[1]])
    print('shifts: ',shifts)
    # Translate bounding boxes to image domain
    boxes = np.multiply(boxes - shifts, scales).astype(np.int32)
    print('boxes: ',boxes.shape,' ',boxes)
    # Filter out detections with zero area. Often only happens in early
    # stages of training when the network weights are still a bit random.
    exclude_ix = np.where(
        (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1]) <= 0)[0]
    if exclude_ix.shape[0] > 0:
        boxes = np.delete(boxes, exclude_ix, axis=0)
        class_ids = np.delete(class_ids, exclude_ix, axis=0)
        scores = np.delete(scores, exclude_ix, axis=0)
        masks = np.delete(masks, exclude_ix, axis=0)
        N = class_ids.shape[0]

    # Resize masks to original image size and set boundary threshold.
    full_masks = []
    for i in range(N):
        # Convert neural network mask to full size mask
        full_mask = utils.unmold_mask(masks[i], boxes[i], image_shape)
        full_masks.append(full_mask)
    full_masks = np.stack(full_masks, axis=-1)\
        if full_masks else np.empty((0,) + masks.shape[1:3])

    return boxes, class_ids, scores, full_masks

In [15]:
with tf.gfile.FastGFile(pb_filepath, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
print('Graph loaded.')
testImage =os.path.join(ROOT_DIR,'model','150_256.jpg') #image of the size defined in the config
sess = tf.InteractiveSession()
image = imageio.imread(testImage)
print('Image loaded.')
images = [image]
print("Processing {} images".format(len(images)))
for im in images:
    modellib.log("image", im)
print('RGB image loaded and preprocessed.')
molded_images, image_metas, windows = mold_inputs(images)
print(molded_images.shape)
print('Images meta: ',image_metas)
img_ph = sess.graph.get_tensor_by_name('input_image:0')
print(img_ph)
img_meta_ph = sess.graph.get_tensor_by_name('input_image_meta:0')
print(img_meta_ph)
detectionsT = sess.graph.get_tensor_by_name('output_detections:0')
print('Found ',detectionsT)
mrcnn_classT = sess.graph.get_tensor_by_name('output_mrcnn_class:0')
print('Found ',mrcnn_classT)
mrcnn_bboxT = sess.graph.get_tensor_by_name('output_mrcnn_bbox:0')
print('Found ', mrcnn_bboxT)
mrcnn_maskT = sess.graph.get_tensor_by_name('output_mrcnn_mask:0')
print('Found ', mrcnn_maskT)
roisT = sess.graph.get_tensor_by_name('output_rois:0')
print('Found ', roisT)
        
detections = sess.run(detectionsT, feed_dict={img_ph: molded_images, img_meta_ph: image_metas})
#print('Detections: ',detections[0].shape, detections[0])
mrcnn_class = sess.run(mrcnn_classT, feed_dict={img_ph: molded_images, img_meta_ph: image_metas})
#print('Classes: ',mrcnn_class[0].shape, mrcnn_class[0])
mrcnn_bbox = sess.run(mrcnn_bboxT, feed_dict={img_ph: molded_images, img_meta_ph: image_metas})
#print('BBoxes: ',mrcnn_bbox[0].shape, mrcnn_bbox[0])
mrcnn_mask = sess.run(mrcnn_maskT, feed_dict={img_ph: molded_images, img_meta_ph: image_metas})
#print('Masks: ',mrcnn_mask[0].shape )#, outputs1[0])
rois = sess.run(roisT, feed_dict={img_ph: molded_images, img_meta_ph: image_metas})
#print('Rois: ',rois[0].shape, rois[0])

results = []
for i, image in enumerate(images):
    print('Calculating results for image#',i)
    final_rois, final_class_ids, final_scores, final_masks =\
    unmold_detections(detections[i], mrcnn_mask[i],
                                    image.shape, windows[i])
    results.append({
        "rois": final_rois,
        "class_ids": final_class_ids,
        "scores": final_scores,
        "masks": final_masks,
    })
r = results[0]
#print(r)
print (r['scores'][0])
print (r['class_ids'][0])
print (r['rois'][0])
print (r['masks'][0].shape)

class_names = ["BG","nuclei"]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'], ax=get_ax())
print('Done')

Graph loaded.
Image loaded.
Processing 1 images
image            shape: (256, 256, 3)         min:       2.00  max:     251.00
RGB image loaded and preprocessed.
IMAGE_PADDING:  True
(256, 256, 3)
Image resized at:  (512, 512, 3)
(0, 0, 512, 512)
2.0
Image molded
Meta of image prepared
(1, 512, 512, 3)
Images meta:  [[  0 256 256   3   0   0 512 512   0   0]]
Tensor("input_image:0", shape=(?, 512, 512, 3), dtype=float32)
Tensor("input_image_meta:0", shape=(?, ?), dtype=float32)
Found  Tensor("output_detections:0", shape=(1, 50, 6), dtype=float32)
Found  Tensor("output_mrcnn_class:0", shape=(?, 1000, 2), dtype=float32)
Found  Tensor("output_mrcnn_bbox:0", shape=(?, 1000, 2, 4), dtype=float32)
Found  Tensor("output_mrcnn_mask:0", shape=(?, 50, 28, 28, 2), dtype=float32)
Found  Tensor("output_rois:0", shape=(1, ?, ?), dtype=float32)


FailedPreconditionError: Attempting to use uninitialized value bn5b_branch2a/moving_mean
	 [[Node: bn5b_branch2a/moving_mean/read = Identity[T=DT_FLOAT, _class=["loc:@bn5b_branch2a/moving_mean"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](bn5b_branch2a/moving_mean)]]
	 [[Node: mrcnn_detection/map/while/strided_slice/_139 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2386_mrcnn_detection/map/while/strided_slice", tensor_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](^_cloopmrcnn_detection/map/while/PadV2/constant_values/_1)]]

Caused by op 'bn5b_branch2a/moving_mean/read', defined at:
  File "c:\programdata\anaconda3\envs\maskrcnn\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\programdata\anaconda3\envs\maskrcnn\lib\runpy.py", line 85, in _run_code
    exec(code, 